# Rent by SA2

## Import Library

In [ ]:
import  pandas as pd
import geopandas as gpd
import folium
import json

## Read data

In [ ]:
sa2_rent_data = pd.read_csv('../../data/curated/merged_final_geo.csv')

In [ ]:
shp_sa2 = gpd.read_file('../../data/landing/region_data/sa2_dataset/sa2_unzip/SA2_2021_AUST_GDA2020.shp')
victoria_sa2 = shp_sa2[shp_sa2['STE_NAME21'] == 'Victoria']

In [ ]:
victoria_sa2

In [ ]:
sa2_rent_data = sa2_rent_data[['region', 'cost_text']]

## visualization

In [ ]:
sa2_rent_data_grouped = sa2_rent_data.groupby('region', as_index=False)['cost_text'].median()

In [ ]:
sa2_rent_data_grouped.columns

In [ ]:
sa2_rent_data_grouped

In [ ]:
victoria_sa2 = victoria_sa2.merge(sa2_rent_data_grouped, left_on='SA2_NAME21', right_on='region', how='left').dropna()

In [ ]:
victoria_sa2 = victoria_sa2.loc[:, ~victoria_sa2.columns.duplicated()]

In [ ]:
victoria_sa2

In [ ]:
m = folium.Map(location=[-37.81, 144.96], zoom_start=6, tiles="cartodbpositron")
geojson_data = json.loads(victoria_sa2.to_json())

folium.Choropleth(
    geo_data=geojson_data,
    name='choropleth',
    data=victoria_sa2,
    columns=['SA2_NAME21', 'cost_text'],
    key_on='feature.properties.SA2_NAME21',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Median Rent Cost'
).add_to(m)

top_10_regions = sa2_rent_data_grouped.nlargest(10, 'cost_text')

for idx, row in top_10_regions.iterrows():
    region_name = row['region']
    avg_cost = row['cost_text']
    
    matching_row = victoria_sa2[victoria_sa2['SA2_NAME21'] == region_name]
    if not matching_row.empty:
        centroid = matching_row.geometry.centroid.iloc[0]
        
        folium.Marker(
            location=[centroid.y, centroid.x],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 12px; color: black; padding: 2px;">{region_name}: {avg_cost:.2f}</div>'
            )
        ).add_to(m)

m.save('../../plots/geo(html)/Victoria_Median_Rent_Cost_SA2.html')